In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error


# No moving average

To allow for "future predicting", stagger data by one week, and use 1 week moving average in conjunction with actual data

In [ ]:
# Retrieve data
df = pd.read_csv('../data/Model Data With MA.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df = df.loc['3/31/20':]
df = df.dropna()
df.head()

# Set Modeling Data
# 1 week delay
# wk1_delay_df = df.copy()[['Ridership', 'Crime', 'temp', 'precip', 'Daily_CPI', 'Gas_Price']].shift(periods=7, fill_value=0)
wk1_delay_df = df.copy()[['Ridership', 'Crime', 'temp', 'precip', 'Daily_CPI', 'Gas_Price']]
# wk1_delay_df = df.copy()[['Ridership 1week MA', 'Crime 1week MA', 'temp 1week MA', 'precip 1week MA''''''''''''''''''''''''''''''''''''''','''''''''''''''''''''''''''''''''''''' 'CPI 1week MA', 'Gas 1week MA']].shift(periods=7, fill_value=0)
wk1_delay_df.index = wk1_delay_df.index + pd.DateOffset(days=7)
wk1_delay_df['target_ridership'] = df['Ridership']

# test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-01-01') & (wk1_delay_df['Date'] < '2021-02-14')]
# new_test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-02-14') & (wk1_delay_df['Date'] < '2021-03-01')]
# wk1_delay_df = wk1_delay_df.append(df.tail(7)[['Ridership', 'Crime', 'temp', 'precip', 'Daily_CPI', 'Gas_Price']])
wk1_delay_df = wk1_delay_df.fillna(0)
wk1_delay_df.tail(14)

In [ ]:
wk1_delay_df.head(14)

Future prediction simulation - 
Use 2021 to 2022 data to "draw out" predictions for Feb 14 to Feb 27, 2022.
Then use the same model to "draw out" predictions for Feb 14 to Feb 27, 2023.

Compare the actual ridership vs predicted ones.

UPDATE - initially, only stagger and do not use moving averages just yet, do that for the second run

In [ ]:

# test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-01-01') & (wk1_delay_df['Date'] < '2021-02-14')]
# new_test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-02-14') & (wk1_delay_df['Date'] < '2021-03-01')]
test1_df = wk1_delay_df.loc['2021-01-01':'2022-02-21']
new_test1_df = wk1_delay_df.loc['2022-02-21':'2022-02-28']

# # Encode the label using LabelEncoder
# X = df.loc[:, 'Ridership':].values
# y = df.loc[:, 'Ridership'].values

# Encode the label using LabelEncoder
X = test1_df.loc[:, 'Ridership':'Gas_Price'].values
# X = test1_df.loc[:, :'Gas 1week MA'].values
y = test1_df.loc[:, 'target_ridership'].values


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Define the pipeline with the hyperparameters to search
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor()
)

# Define the hyperparameter grid to search over
param_grid = {
    'randomforestregressor__n_estimators': [100, 500],
    'randomforestregressor__max_depth': [5, 10]
}

# Create a GridSearchCV object with the pipeline and hyperparameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=10)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Make predictions on the test data using the best model found by GridSearchCV
y_pred = grid_search.predict(X_test)


# Use the best model to predict on new data
new_X = new_test1_df.loc[:, 'Ridership':'Gas_Price'].values
# new_X = new_test1_df.loc[:, :'Gas 1week MA'].values
new_y_pred = grid_search.predict(new_X)

# Add the predicted ridership values to the original dataframe
new_test1_df['Predicted Ridership'] = new_y_pred

# Print the best hyperparameters and test accuracy
print("Best hyperparameters: ", grid_search.best_params_)
print("Test accuracy: ", grid_search.score(X_test, y_test))
print("Test accuracy new_y_pred: ", grid_search.score(new_X, new_y_pred))

# new_test1_df['Predicted Ridership'] = pd.Series(new_y_pred)

# Print the new dataframe with predicted ridership values
new_test1_df
wk1_delay_df
# new_test1_df['actual_ridership'] = df['Ridership'].loc['2022-02-21':'2022-02-28']
# new_test1_df['actual_ridership'] = wk1_delay_df['target_ridership'].loc['2022-02-21':'2022-02-28']
df.loc['2022-02-21':'2022-02-28']
new_test1_df
new_test1_df[['target_ridership','Predicted Ridership']]

In [ ]:

# get the target and predicted values
target = new_test1_df['target_ridership']
predicted = new_test1_df['Predicted Ridership']

# calculate MSE and RMSE
mse = mean_squared_error(target, predicted)
rmse = mean_squared_error(target, predicted, squared=False)

print("MSE:", mse)
print("RMSE:", rmse)

Part 2


In [ ]:
# test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-01-01') & (wk1_delay_df['Date'] < '2021-02-14')]
# new_test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-02-14') & (wk1_delay_df['Date'] < '2021-03-01')]
test2_df = wk1_delay_df.loc['2021-01-01':'2022-12-31']
new_test2_df = wk1_delay_df.loc['2023-01-01':'2023-01-07']


# # Encode the label using LabelEncoder
# X = df.loc[:, 'Ridership':].values
# y = df.loc[:, 'Ridership'].values

# Encode the label using LabelEncoder
X = test2_df.loc[:, 'Ridership':'Gas_Price'].values
y = test2_df.loc[:, 'target_ridership'].values


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Define the pipeline with the hyperparameters to search
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor()
)

# Define the hyperparameter grid to search over
param_grid = {
    'randomforestregressor__n_estimators': [100, 500],
    'randomforestregressor__max_depth': [5, 10]
}

# Create a GridSearchCV object with the pipeline and hyperparameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=10)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Make predictions on the test data using the best model found by GridSearchCV
y_pred = grid_search.predict(X_test)

# Print the best hyperparameters and test accuracy
print("Best hyperparameters: ", grid_search.best_params_)
print("Test accuracy: ", grid_search.score(X_test, y_test))


# Use the best model to predict on new data
new_X = new_test2_df.loc[:, 'Ridership':'Gas_Price'].values
new_y_pred = grid_search.predict(new_X)

# Add the predicted ridership values to the original dataframe
new_test2_df['Predicted Ridership'] = new_y_pred
# new_test1_df['Predicted Ridership'] = pd.Series(new_y_pred)

# Print the new dataframe with predicted ridership values
new_test2_df
new_test2_df[['target_ridership','Predicted Ridership']]

# With raw data and weekly moving averages

In [ ]:
# Retrieve data
df = pd.read_csv('../data/Model Data With MA.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df = df.loc['3/31/20':]
df = df.dropna()
df.head()

# Set Modeling Data
# 1 week delay
wk1_delay_df = pd.DataFrame
wk1_delay_df = df.copy()[['Ridership','Crime','temp','precip','Daily_CPI','Gas_Price',
                          'Ridership 1week MA', 'Crime 1week MA', 'temp 1week MA', 'precip 1week MA', 'CPI 1week MA', 'Gas 1week MA']]
# wk1_delay_df = df.copy()[['Ridership','Crime','temp','precip','Daily_CPI','Gas_Price',
#                           'Ridership 1week MA', 'Crime 1week MA', 'temp 1week MA', 'precip 1week MA', 'CPI 1week MA', 'Gas 1week MA']].shift(periods=7, fill_value=0)
wk1_delay_df.index = wk1_delay_df.index + pd.DateOffset(days=7)
# wk1_delay_df = df.copy()[['Ridership','Crime','temp','precip','Daily_CPI','Gas_Price',
#                           'Ridership 1week MA', 'Crime 1week MA', 'temp 1week MA', 'precip 1week MA', 'CPI 1week MA', 'Gas 1week MA']]#.shift(periods=7, fill_value=0)
wk1_delay_df['target_ridership'] = df['Ridership']# wk1_delay_df
wk1_delay_df = wk1_delay_df.append(df.tail(7)[['Ridership','Crime','temp','precip','Daily_CPI','Gas_Price',
                          'Ridership 1week MA', 'Crime 1week MA', 'temp 1week MA', 'precip 1week MA', 'CPI 1week MA', 'Gas 1week MA']])
wk1_delay_df = wk1_delay_df.fillna(0)

wk1_delay_df


In [ ]:

# test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-01-01') & (wk1_delay_df['Date'] < '2021-02-14')]
# new_test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-02-14') & (wk1_delay_df['Date'] < '2021-03-01')]
test1_df = wk1_delay_df.loc['2021-01-01':'2022-02-21']
new_test1_df = wk1_delay_df.loc['2022-02-21':'2022-02-28']

# # Encode the label using LabelEncoder
# X = df.loc[:, 'Ridership':].values
# y = df.loc[:, 'Ridership'].values

# Encode the label using LabelEncoder
X = test1_df.loc[:, :'Gas 1week MA'].values
# X = test1_df.loc[:, :'Gas 1week MA'].values
y = test1_df.loc[:, 'target_ridership'].values


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Define the pipeline with the hyperparameters to search
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor()
)

# Define the hyperparameter grid to search over
param_grid = {
    'randomforestregressor__n_estimators': [100, 500],
    'randomforestregressor__max_depth': [5, 10]
}

# Create a GridSearchCV object with the pipeline and hyperparameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=10)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Make predictions on the test data using the best model found by GridSearchCV
y_pred = grid_search.predict(X_test)

# Use the best model to predict on new data
# new_X = new_test1_df.loc[:, 'Ridership':'Gas_Price'].values
new_X = new_test1_df.loc[:, :'Gas 1week MA'].values
new_y_pred = grid_search.predict(new_X)

# Print the best hyperparameters and test accuracy
print("Best hyperparameters: ", grid_search.best_params_)
print("Test accuracy: ", grid_search.score(X_test, y_test))
print("Test accuracy new_y_pred: ", grid_search.score(new_X, new_y_pred))

# Add the predicted ridership values to the original dataframe
new_test1_df['Predicted Ridership'] = new_y_pred
# new_test1_df['Predicted Ridership'] = pd.Series(new_y_pred)

# Print the new dataframe with predicted ridership values
new_test1_df
new_test1_df[['target_ridership','Predicted Ridership']]

In [ ]:
# test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-01-01') & (wk1_delay_df['Date'] < '2021-02-14')]
# new_test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-02-14') & (wk1_delay_df['Date'] < '2021-03-01')]
test2_df = wk1_delay_df.loc['2021-01-01':'2022-12-31']
new_test2_df = wk1_delay_df.loc['2023-01-01':'2023-01-07']


# # Encode the label using LabelEncoder
# X = df.loc[:, 'Ridership':].values
# y = df.loc[:, 'Ridership'].values

# Encode the label using LabelEncoder
X = test2_df.loc[:, :'Gas 1week MA'].values
y = test2_df.loc[:, 'target_ridership'].values


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Define the pipeline with the hyperparameters to search
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor()
)

# Define the hyperparameter grid to search over
param_grid = {
    'randomforestregressor__n_estimators': [100, 500],
    'randomforestregressor__max_depth': [5, 10]
}

# Create a GridSearchCV object with the pipeline and hyperparameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=10)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Make predictions on the test data using the best model found by GridSearchCV
y_pred = grid_search.predict(X_test)

# Print the best hyperparameters and test accuracy
print("Best hyperparameters: ", grid_search.best_params_)
print("Test accuracy: ", grid_search.score(X_test, y_test))


# Use the best model to predict on new data
new_X = new_test2_df.loc[:, :'Gas 1week MA'].values
new_y_pred = grid_search.predict(new_X)

# Add the predicted ridership values to the original dataframe
new_test2_df['Predicted Ridership'] = new_y_pred
# new_test1_df['Predicted Ridership'] = pd.Series(new_y_pred)

# Print the new dataframe with predicted ridership values
new_test2_df
new_test2_df[['target_ridership','Predicted Ridership']]

In [ ]:
new_test2_df.loc[:, :'Gas 1week MA']

In [ ]:
new_test2_df

# only 1wk ma

In [ ]:
# Retrieve data
df = pd.read_csv('../data/Model Data With MA.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df = df.loc['3/31/20':]
df = df.dropna()
df.head()

# Set Modeling Data
# 1 week delay
wk1_delay_df = pd.DataFrame
# wk1_delay_df = df.copy()[['Ridership 1week MA', 'Crime 1week MA', 'temp 1week MA', 'precip 1week MA', 'CPI 1week MA', 'Gas 1week MA']].shift(periods=7, fill_value=0)
wk1_delay_df = df.copy()[['Ridership 1week MA', 'Crime 1week MA', 'temp 1week MA', 'precip 1week MA', 'CPI 1week MA', 'Gas 1week MA']]
wk1_delay_df.index = wk1_delay_df.index + pd.DateOffset(days=7)
wk1_delay_df['target_ridership'] = df['Ridership']
wk1_delay_df = wk1_delay_df.append(df.tail(7)[['Ridership 1week MA', 'Crime 1week MA', 'temp 1week MA', 'precip 1week MA', 'CPI 1week MA', 'Gas 1week MA']])
# wk1_delay_df
wk1_delay_df = wk1_delay_df.fillna(0)
wk1_delay_df


In [ ]:

# test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-01-01') & (wk1_delay_df['Date'] < '2021-02-14')]
# new_test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-02-14') & (wk1_delay_df['Date'] < '2021-03-01')]
test1_df = wk1_delay_df.loc['2021-01-01':'2022-02-21']
new_test1_df = wk1_delay_df.loc['2022-02-21':'2022-02-28']


# # Encode the label using LabelEncoder
# X = df.loc[:, 'Ridership':].values
# y = df.loc[:, 'Ridership'].values

# Encode the label using LabelEncoder
# X = test1_df.loc[:, 'Ridership':'Gas_Price'].values
X = test1_df.loc[:, :'Gas 1week MA'].values
y = test1_df.loc[:, 'target_ridership'].values


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Define the pipeline with the hyperparameters to search
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor()
)

# Define the hyperparameter grid to search over
param_grid = {
    'randomforestregressor__n_estimators': [100, 500],
    'randomforestregressor__max_depth': [5, 10]
}

# Create a GridSearchCV object with the pipeline and hyperparameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=10)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Make predictions on the test data using the best model found by GridSearchCV
y_pred = grid_search.predict(X_test)

# Use the best model to predict on new data
# new_X = new_test1_df.loc[:, 'Ridership':'Gas_Price'].values
new_X = new_test1_df.loc[:, :'Gas 1week MA'].values
new_y_pred = grid_search.predict(new_X)

# Print the best hyperparameters and test accuracy
print("Best hyperparameters: ", grid_search.best_params_)
print("Test accuracy: ", grid_search.score(X_test, y_test))
print("Test accuracy new_y_pred: ", grid_search.score(new_X, new_y_pred))


# Add the predicted ridership values to the original dataframe
new_test1_df['Predicted Ridership'] = new_y_pred
# new_test1_df['Predicted Ridership'] = pd.Series(new_y_pred)

# Print the new dataframe with predicted ridership values
new_test1_df
new_test1_df[['target_ridership','Predicted Ridership']]

In [ ]:
# test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-01-01') & (wk1_delay_df['Date'] < '2021-02-14')]
# new_test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-02-14') & (wk1_delay_df['Date'] < '2021-03-01')]
test2_df = wk1_delay_df.loc['2021-01-01':'2022-12-31']
new_test2_df = wk1_delay_df.loc['2023-01-01':'2023-01-07']


# # Encode the label using LabelEncoder
# X = df.loc[:, 'Ridership':].values
# y = df.loc[:, 'Ridership'].values

# Encode the label using LabelEncoder
X = test2_df.loc[:, :'Gas 1week MA'].values
y = test2_df.loc[:, 'target_ridership'].values


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Define the pipeline with the hyperparameters to search
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor()
)

# Define the hyperparameter grid to search over
param_grid = {
    'randomforestregressor__n_estimators': [100, 500],
    'randomforestregressor__max_depth': [5, 10]
}

# Create a GridSearchCV object with the pipeline and hyperparameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=10)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Make predictions on the test data using the best model found by GridSearchCV
y_pred = grid_search.predict(X_test)

# Use the best model to predict on new data
new_X = new_test2_df.loc[:, :'Gas 1week MA'].values
new_y_pred = grid_search.predict(new_X)

# Print the best hyperparameters and test accuracy
print("Best hyperparameters: ", grid_search.best_params_)
print("Test accuracy: ", grid_search.score(X_test, y_test))
print("Test accuracy new_y_pred: ", grid_search.score(new_X, new_y_pred))


# Add the predicted ridership values to the original dataframe
new_test2_df['Predicted Ridership'] = new_y_pred
# new_test1_df['Predicted Ridership'] = pd.Series(new_y_pred)

# Print the new dataframe with predicted ridership values
new_test2_df
new_test2_df[['target_ridership','Predicted Ridership']]

# raw data + all moving averages

In [47]:

# Retrieve data
df = pd.read_csv('../data/Model Data With MA.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df = df.loc['3/31/20':]
df = df.dropna()
df.head()

# Set Modeling Data
# 1 week delay
wk1_delay_df = pd.DataFrame
# wk1_delay_df = df.copy()[['Ridership','Crime','temp','precip','Daily_CPI','Gas_Price','Ridership 1week MA','Crime 1week MA','temp 1week MA','precip 1week MA','CPI 1week MA','Gas 1week MA','Ridership 2weeks MA','Crime 2weeks MA','temp 2weeks MA','precip 2weeks MA','CPI 2weeks MA','Gas 2weeks MA','Ridership 1Month MA','Crime 1Month MA','temp 1Month MA','precip 1Month MA','CPI 1Month MA','Gas 1Month MA','Ridership 3Months MA','Crime 3Months MA','temp 3Months MA','precip 3Months MA','CPI 3Months MA','Gas 3Months MA']].shift(periods=7, fill_value=0)
wk1_delay_df = df.copy()[['Ridership','Crime','temp','precip','Daily_CPI','Gas_Price','Ridership 1week MA','Crime 1week MA','temp 1week MA','precip 1week MA','CPI 1week MA','Gas 1week MA','Ridership 2weeks MA','Crime 2weeks MA','temp 2weeks MA','precip 2weeks MA','CPI 2weeks MA','Gas 2weeks MA','Ridership 1Month MA','Crime 1Month MA','temp 1Month MA','precip 1Month MA','CPI 1Month MA','Gas 1Month MA','Ridership 3Months MA','Crime 3Months MA','temp 3Months MA','precip 3Months MA','CPI 3Months MA','Gas 3Months MA']]
wk1_delay_df.index = wk1_delay_df.index + pd.DateOffset(days=7)
wk1_delay_df['target_ridership'] = df['Ridership']
wk1_delay_df = wk1_delay_df.append(df.tail(7)[['Ridership','Crime','temp','precip','Daily_CPI','Gas_Price','Ridership 1week MA','Crime 1week MA','temp 1week MA','precip 1week MA','CPI 1week MA','Gas 1week MA','Ridership 2weeks MA','Crime 2weeks MA','temp 2weeks MA','precip 2weeks MA','CPI 2weeks MA','Gas 2weeks MA','Ridership 1Month MA','Crime 1Month MA','temp 1Month MA','precip 1Month MA','CPI 1Month MA','Gas 1Month MA','Ridership 3Months MA','Crime 3Months MA','temp 3Months MA','precip 3Months MA','CPI 3Months MA','Gas 3Months MA']])
# wk1_delay_df
wk1_delay_df = wk1_delay_df.fillna(0)
wk1_delay_df


C:\Users\cjong\AppData\Local\Temp\ipykernel_3324\3612841503.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wk1_delay_df = wk1_delay_df.append(df.tail(7)[['Ridership','Crime','temp','precip','Daily_CPI','Gas_Price','Ridership 1week MA','Crime 1week MA','temp 1week MA','precip 1week MA','CPI 1week MA','Gas 1week MA','Ridership 2weeks MA','Crime 2weeks MA','temp 2weeks MA','precip 2weeks MA','CPI 2weeks MA','Gas 2weeks MA','Ridership 1Month MA','Crime 1Month MA','temp 1Month MA','precip 1Month MA','CPI 1Month MA','Gas 1Month MA','Ridership 3Months MA','Crime 3Months MA','temp 3Months MA','precip 3Months MA','CPI 3Months MA','Gas 3Months MA']])


,Ridership,Crime,temp,precip,Daily_CPI,Gas_Price,Ridership 1week MA,Crime 1week MA,temp 1week MA,precip 1week MA,...,precip 1Month MA,CPI 1Month MA,Gas 1Month MA,Ridership 3Months MA,Crime 3Months MA,temp 3Months MA,precip 3Months MA,CPI 3Months MA,Gas 3Months MA,target_ridership
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-06,426737,1139,74.1,0.001,0.004159,2.17,5.410310e+05,1094.714286,66.842857,0.000429,...,0.025467,0.010144,2.168333,1.089713e+06,972.222222,53.040000,0.036122,-0.000744,2.324000,464198.0
2020-06-07,344426,1188,65.7,0.000,0.003746,2.17,5.445183e+05,1132.857143,67.657143,0.000429,...,0.023200,0.009731,2.167333,1.034319e+06,971.466667,53.232222,0.036022,-0.000627,2.319111,377990.0
2020-06-08,687162,1928,63.2,0.000,0.003333,2.18,5.864103e+05,1266.142857,67.742857,0.000429,...,0.023167,0.009318,2.166667,9.810525e+05,978.433333,53.357778,0.035856,-0.000511,2.314333,804542.0
2020-06-09,693760,1407,66.7,0.000,0.003760,2.18,5.916951e+05,1306.285714,68.057143,0.000429,...,0.022167,0.008934,2.166000,9.276631e+05,979.533333,53.538889,0.035744,-0.000386,2.309556,842306.0
2020-06-10,681714,934,69.9,0.366,0.004186,2.18,5.927307e+05,1276.000000,68.685714,0.052714,...,0.034367,0.008577,2.166333,8.741660e+05,975.744444,53.807778,0.039811,-0.000253,2.304778,857015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,2729514,1130,31.6,0.000,0.022685,3.45,2.350087e+06,1067.142857,28.528571,0.187714,...,0.165633,0.013758,3.664667,3.045546e+06,1398.400000,49.344444,0.136144,0.008318,3.715556,0.0
2022-12-28,2947028,1173,40.1,0.000,0.023309,3.45,2.262058e+06,1059.285714,29.414286,0.187714,...,0.165633,0.014319,3.652667,3.037093e+06,1395.433333,49.121111,0.136144,0.008539,3.712556,0.0
2022-12-29,3039432,1158,45.3,0.000,0.023933,3.45,2.240965e+06,1046.714286,30.142857,0.157571,...,0.165633,0.014901,3.640667,3.031940e+06,1391.877778,48.981111,0.136144,0.008768,3.709556,0.0


In [48]:

# test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-01-01') & (wk1_delay_df['Date'] < '2021-02-14')]
# new_test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-02-14') & (wk1_delay_df['Date'] < '2021-03-01')]
test1_df = wk1_delay_df.loc['2021-01-01':'2022-02-21']
# new_test1_df = wk1_delay_df.loc['2022-02-21':'2022-02-28']
new_test1_df = wk1_delay_df.loc['2022-12-01':'2022-12-07']


# # Encode the label using LabelEncoder
# X = df.loc[:, 'Ridership':].values
# y = df.loc[:, 'Ridership'].values

# Encode the label using LabelEncoder
# X = test1_df.loc[:, 'Ridership':'Gas_Price'].values
X = test1_df.loc[:, :'Gas 3Months MA'].values
y = test1_df.loc[:, 'target_ridership'].values


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Define the pipeline with the hyperparameters to search
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor()
)

# Define the hyperparameter grid to search over
param_grid = {
    'randomforestregressor__n_estimators': [100, 500],
    'randomforestregressor__max_depth': [5, 10]
}

# Create a GridSearchCV object with the pipeline and hyperparameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=10)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Make predictions on the test data using the best model found by GridSearchCV
y_pred = grid_search.predict(X_test)

# Use the best model to predict on new data
# new_X = new_test1_df.loc[:, 'Ridership':'Gas_Price'].values
new_X = new_test1_df.loc[:, :'Gas 3Months MA'].values
new_y_pred = grid_search.predict(new_X)

# Print the best hyperparameters and test accuracy
print("Best hyperparameters: ", grid_search.best_params_)
print("Test accuracy: ", grid_search.score(X_test, y_test))
print("Test accuracy new_y_pred: ", grid_search.score(new_X, new_y_pred))


# Add the predicted ridership values to the original dataframe
new_test1_df['Predicted Ridership'] = new_y_pred
# new_test1_df['Predicted Ridership'] = pd.Series(new_y_pred)

# Print the new dataframe with predicted ridership values
new_test1_df
new_test1_df[['target_ridership','Predicted Ridership']]

####


# Use the best model to predict on new data
new_X = new_test1_df.loc[:, :'Gas 3Months MA'].values
new_y_pred = grid_search.predict(new_X)

# Add the predicted ridership values to the original dataframe
new_test1_df['Predicted Ridership'] = new_y_pred
# new_test1_df['Predicted Ridership'] = pd.Series(new_y_pred)

# Print the new dataframe with predicted ridership values
new_test1_df
new_test1_df[['target_ridership','Predicted Ridership']]

Best hyperparameters:  {'randomforestregressor__max_depth': 10, 'randomforestregressor__n_estimators': 500}
Test accuracy:  0.8409632586429279
Test accuracy new_y_pred:  1.0


,target_ridership,Predicted Ridership
Date,,
2022-12-01,3770495.0,2.165088e+06
2022-12-02,3600504.0,2.410609e+06
2022-12-03,2291687.0,2.326445e+06
2022-12-04,1990479.0,2.154110e+06
2022-12-05,3481640.0,2.994218e+06
2022-12-06,3753299.0,2.617626e+06
2022-12-07,3835333.0,2.614956e+06


In [49]:
# test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-01-01') & (wk1_delay_df['Date'] < '2021-02-14')]
# new_test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-02-14') & (wk1_delay_df['Date'] < '2021-03-01')]
test2_df = wk1_delay_df.loc['2021-01-01':'2022-12-31']
new_test2_df = wk1_delay_df.loc['2023-01-01':'2023-01-07']


# # Encode the label using LabelEncoder
# X = df.loc[:, 'Ridership':].values
# y = df.loc[:, 'Ridership'].values

# Encode the label using LabelEncoder
X = test2_df.loc[:, :'Gas 3Months MA'].values
y = test2_df.loc[:, 'target_ridership'].values


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Define the pipeline with the hyperparameters to search
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor()
)

# Define the hyperparameter grid to search over
param_grid = {
    'randomforestregressor__n_estimators': [100, 500],
    'randomforestregressor__max_depth': [5, 10]
}

# Create a GridSearchCV object with the pipeline and hyperparameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=10)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Make predictions on the test data using the best model found by GridSearchCV
y_pred = grid_search.predict(X_test)

# Print the best hyperparameters and test accuracy
print("Best hyperparameters: ", grid_search.best_params_)
print("Test accuracy: ", grid_search.score(X_test, y_test))


# Use the best model to predict on new data
new_X = new_test2_df.loc[:, :'Gas 3Months MA'].values
new_y_pred = grid_search.predict(new_X)

# Add the predicted ridership values to the original dataframe
new_test2_df['Predicted Ridership'] = new_y_pred
# new_test1_df['Predicted Ridership'] = pd.Series(new_y_pred)

# Print the new dataframe with predicted ridership values
new_test2_df
new_test2_df[['target_ridership','Predicted Ridership']]

Best hyperparameters:  {'randomforestregressor__max_depth': 10, 'randomforestregressor__n_estimators': 100}
Test accuracy:  0.8444363165317752


,target_ridership,Predicted Ridership
Date,,
2023-01-01,0.0,5.267077e+05
2023-01-02,0.0,6.732785e+05
2023-01-03,0.0,1.115323e+06
2023-01-04,0.0,7.622824e+05
2023-01-05,0.0,7.095374e+05
2023-01-06,0.0,7.751920e+05
2023-01-07,0.0,6.967807e+05


# Moving Averages only

In [ ]:

# Retrieve data
df = pd.read_csv('../data/Model Data With MA.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df = df.loc['3/31/20':]
df = df.dropna()
df.head()

# Set Modeling Data
# 1 week delay
wk1_delay_df = pd.DataFrame
# wk1_delay_df = df.copy()[['Ridership','Crime','temp','precip','Daily_CPI','Gas_Price','Ridership 1week MA','Crime 1week MA','temp 1week MA','precip 1week MA','CPI 1week MA','Gas 1week MA','Ridership 2weeks MA','Crime 2weeks MA','temp 2weeks MA','precip 2weeks MA','CPI 2weeks MA','Gas 2weeks MA','Ridership 1Month MA','Crime 1Month MA','temp 1Month MA','precip 1Month MA','CPI 1Month MA','Gas 1Month MA','Ridership 3Months MA','Crime 3Months MA','temp 3Months MA','precip 3Months MA','CPI 3Months MA','Gas 3Months MA']].shift(periods=7, fill_value=0)
wk1_delay_df = df.copy()[['Ridership 1week MA','Crime 1week MA','temp 1week MA','precip 1week MA','CPI 1week MA','Gas 1week MA','Ridership 2weeks MA','Crime 2weeks MA','temp 2weeks MA','precip 2weeks MA','CPI 2weeks MA','Gas 2weeks MA','Ridership 1Month MA','Crime 1Month MA','temp 1Month MA','precip 1Month MA','CPI 1Month MA','Gas 1Month MA','Ridership 3Months MA','Crime 3Months MA','temp 3Months MA','precip 3Months MA','CPI 3Months MA','Gas 3Months MA']]
wk1_delay_df.index = wk1_delay_df.index + pd.DateOffset(days=7)
wk1_delay_df['target_ridership'] = df['Ridership']
wk1_delay_df = wk1_delay_df.append(df.tail(7)[['Ridership 1week MA','Crime 1week MA','temp 1week MA','precip 1week MA','CPI 1week MA','Gas 1week MA','Ridership 2weeks MA','Crime 2weeks MA','temp 2weeks MA','precip 2weeks MA','CPI 2weeks MA','Gas 2weeks MA','Ridership 1Month MA','Crime 1Month MA','temp 1Month MA','precip 1Month MA','CPI 1Month MA','Gas 1Month MA','Ridership 3Months MA','Crime 3Months MA','temp 3Months MA','precip 3Months MA','CPI 3Months MA','Gas 3Months MA']])
# wk1_delay_df
wk1_delay_df = wk1_delay_df.fillna(0)
wk1_delay_df


In [ ]:

# test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-01-01') & (wk1_delay_df['Date'] < '2021-02-14')]
# new_test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-02-14') & (wk1_delay_df['Date'] < '2021-03-01')]
test1_df = wk1_delay_df.loc['2021-01-01':'2022-02-21']
new_test1_df = wk1_delay_df.loc['2022-02-21':'2022-02-28']


# # Encode the label using LabelEncoder
# X = df.loc[:, 'Ridership':].values
# y = df.loc[:, 'Ridership'].values

# Encode the label using LabelEncoder
# X = test1_df.loc[:, 'Ridership':'Gas_Price'].values
X = test1_df.loc[:, :'Gas 3Months MA'].values
y = test1_df.loc[:, 'target_ridership'].values


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Define the pipeline with the hyperparameters to search
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor()
)

# Define the hyperparameter grid to search over
param_grid = {
    'randomforestregressor__n_estimators': [100, 500],
    'randomforestregressor__max_depth': [5, 10]
}

# Create a GridSearchCV object with the pipeline and hyperparameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=10)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Make predictions on the test data using the best model found by GridSearchCV
y_pred = grid_search.predict(X_test)

# Use the best model to predict on new data
# new_X = new_test1_df.loc[:, 'Ridership':'Gas_Price'].values
new_X = new_test1_df.loc[:, :'Gas 3Months MA'].values
new_y_pred = grid_search.predict(new_X)

# Print the best hyperparameters and test accuracy
print("Best hyperparameters: ", grid_search.best_params_)
print("Test accuracy: ", grid_search.score(X_test, y_test))
print("Test accuracy new_y_pred: ", grid_search.score(new_X, new_y_pred))


# Add the predicted ridership values to the original dataframe
new_test1_df['Predicted Ridership'] = new_y_pred
# new_test1_df['Predicted Ridership'] = pd.Series(new_y_pred)

# Print the new dataframe with predicted ridership values
new_test1_df
new_test1_df[['target_ridership','Predicted Ridership']]

In [ ]:
# test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-01-01') & (wk1_delay_df['Date'] < '2021-02-14')]
# new_test1_df = wk1_delay_df[(wk1_delay_df['Date'] >= '2021-02-14') & (wk1_delay_df['Date'] < '2021-03-01')]
test2_df = wk1_delay_df.loc['2022-01-01':'2022-12-31']
new_test2_df = wk1_delay_df.loc['2023-01-01':'2023-01-07']


# # Encode the label using LabelEncoder
# X = df.loc[:, 'Ridership':].values
# y = df.loc[:, 'Ridership'].values

# Encode the label using LabelEncoder
X = test2_df.loc[:, :'Gas 3Months MA'].values
y = test2_df.loc[:, 'target_ridership'].values


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Define the pipeline with the hyperparameters to search
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor()
)

# Define the hyperparameter grid to search over
param_grid = {
    'randomforestregressor__n_estimators': [100, 500],
    'randomforestregressor__max_depth': [5, 10]
}

# Create a GridSearchCV object with the pipeline and hyperparameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=10)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Make predictions on the test data using the best model found by GridSearchCV
y_pred = grid_search.predict(X_test)

# Print the best hyperparameters and test accuracy
print("Best hyperparameters: ", grid_search.best_params_)
print("Test accuracy: ", grid_search.score(X_test, y_test))


# Use the best model to predict on new data
new_X = new_test2_df.loc[:, :'Gas 3Months MA'].values
new_y_pred = grid_search.predict(new_X)

# Add the predicted ridership values to the original dataframe
new_test2_df['Predicted Ridership'] = new_y_pred
# new_test1_df['Predicted Ridership'] = pd.Series(new_y_pred)

# Print the new dataframe with predicted ridership values
new_test2_df
new_test2_df[['target_ridership','Predicted Ridership']]